<a href="https://colab.research.google.com/github/Loop3D/2022_GA_Workshop/blob/main/template_map2loop_loopstructural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio
!pip install git+https://github.com/geopandas/geopandas.git@v0.10.2
!pip install hjson
!pip install owslib
!pip install git+https://github.com/Loop3D/map2model_cpp.git
!pip install git+https://github.com/Loop3D/LoopProjectFile.git
!pip install pygeos
!pip install mplstereonet
!pip install lavavu-osmesa
!pip install git+https://github.com/Loop3d/LoopStructural@v1.5.3
!pip install git+https://github.com/Loop3d/map2loop-2.git@1.3.1
!pip install beartype
!pip install osgeo

In [ ]:
from google.colab import files

uploaded = files.upload()

geology_filename = "./CombinedPolygons.shp"
fault_filename = "./CombinedFaults.shp"
fold_filename = "./CombinedFaults.shp"
structure_filename = "./StructClip.shp"
mindep_filename = "./null_mindeps.shp"
dtm_filename = "./June22DTM.tif"
metadata_filename = "./m2l_config.hjson"

clut_filename = "./clut.csv"

In [ ]:
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
import time
from datetime import datetime

nowtime=datetime.now().isoformat(timespec='minutes')   
model_name=nowtime.replace(":","-").replace("T","-")

t0 = time.time()

proj = Project( 
                  geology_filename=geology_filename,
                  fault_filename=fault_filename,
                  fold_filename=fold_filename,
                  structure_filename=structure_filename,
                  mindep_filename=mindep_filename,
                  dtm_filename=dtm_filename,
                  metadata_filename=metadata_filename,
                  overwrite="true",
                  verbose_level=VerboseLevel.NONE,
                  project_path=model_name,
                  working_projection="EPSG:28355",
                )

proj.update_config(
                    bbox_3d={
                         "minx": 319518,
                         "miny": 5399467,
                         "maxx": 340530,
                         "maxy": 5420458,
                         "base": -6000,
                         "top": 0,
                    },
                    clut_path=clut_filename,
                    run_flags={
                        'aus': False,
                        'contact_decimate': 5,
                        'intrusion_mode': 1,
                        'min_fault_length': 3000,
                        'use_fat': False,
                        'use_interpolations': False,
                        'drift_prefix':['cover'],
#                         'drift_prefix':['Qha','Qhb','Qhbd','Qhd','Qhdm','Qpso','Qptn','qv','Water','Unknown'],
                    }
                  )
proj.workflow['contact_dips']=False
proj.run()

In [ ]:
vtk_filename=model_name+'/vtk/'+'surface_name_{}.vtk'

fault_params = {'interpolatortype':'FDI',
                 'nelements':1e4,
                }
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                    'nelements':1e5,  # how many tetras/voxels
                    'regularisation':5,
                    }

In [ ]:
t1 = time.time()
import LoopProjectFile as LPF
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer
import lavavu
import numpy as np
import os

LPFilename = os.path.join(proj.config.output_path,"output.loop3d")
projFile = LPF.ProjectFile(LPFilename)
processedData = LPFProcessor(projFile)
processedData.foliation_properties['sg'] = foliation_params
processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
processedData.fault_properties['nelements'] = fault_params['nelements']

model = GeologicalModel(np.append(projFile.origin[:2],projFile.extents['depth'][1]),projFile.extents['utm'][3:5:1] + [projFile.extents['depth'][0]])
model = model.from_processor(processedData)
model.update()

view = LavaVuModelViewer(model,vertical_exaggeration=1) 
view.nsteps=np.array([50,50,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(filename=vtk_filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(filename=vtk_filename,strati=False,displacement_cmap = 'rainbow')
view.lv.webgl(model_name+'/vtk/'+model_name)
view.nsteps = np.array([200,200,200])

view.add_model()

view.lv.control.Range('alpha', label="Global Opacity")
view.lv.control.DualRange(['xmin', 'xmax'], label="x clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['ymin', 'ymax'], label="y clip", step=0.01, values=[0.0,1.0])
view.lv.control.DualRange(['zmin', 'zmax'], label="z clip", step=0.01, values=[0.0,1.0])
view.lv.control.Range(command='background', range=(0,1), step=0.1, value=0.8)
view.lv.control.show() #Show the control panel, including the viewer window
view.interactive()

t2 = time.time()
print("m2l",(t1-t0)/60,"LoopStructuralLPF",(t2-t1)/60,"Total",(t2-t0)/60,"minutes")